In [1]:
#Estatísticos
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

#Manipulação de Dados
import pandas as pd
import numpy as np
from numpy.linalg import LinAlgError

#Utilitários
import pickle
import itertools
from datetime import date, timedelta
from joblib import Parallel, delayed
import warnings

#Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

#Ajustes dos Gráficos
rcParams['figure.figsize'] = 18, 12

from Fase3_Aux_Func import (agrupamento_periodico,
                            ajuste_datas_historica,
                            conversao_array,
                            plot_serie,
                            plot_decomposicao,
                            suavizacao_simples,
                            suavizacao_exponencial,
                            holt_winters,
                            arma,
                            arima_sazonal,
                            analise_residuos,
                            check_stationarity,
                            grid_search_best_model,
                            grid_search_best_model_no_parallel)


# Suprimir todos os warnings
warnings.filterwarnings('ignore')

In [2]:
global local_origem 
global data_corte_treinamento

local_origem = "C:/Users/f112596/Desktop/Projetos/DO/Lab Transp/Dados Processados"
data_corte_treinamento = '2023-04-30'

### 1. Carregramento dos Dados

In [3]:
'''
LEMBRA DE MUDAR O NOME DO ARQUIVO PARA df_final_tratado DEPOIS DE AUTALIZAR O BD_BILHETAGEM E RODAR O DF TRATADO
'''
df_bilhetagem = pd.read_csv(local_origem + "/Dataframe Final.csv", sep=";", decimal="." )
df_bilhetagem['Volume_Passageiros (BU + VT + DIN)'] = df_bilhetagem['Volume_Passageiros (BU + VT + DIN)'].astype(int)
df_bilhetagem.head()

,Sub_Prefeitura,Zona,Ano,Mes,Volume_Passageiros (BU + VT + DIN),Qtde_Linhas
0,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,856,62
1,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2282,62
2,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2793,62
3,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2818,62
4,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,2928,62


In [4]:
'''
Irá realizar os ajustes de datas necessários, periorizando por dia, além de quebrar em 2 arquivos
'''
df_subpref, df_zonas = agrupamento_periodico(df_bilhetagem)

In [5]:
#Criação das Listas
Sub_Prefeituras = np.unique(df_bilhetagem['Sub_Prefeitura'].astype(str).to_list())
Zonas = np.unique(df_bilhetagem['Zona'].astype(str).to_list())

### 2. Ajuste dos Modelos (Grid Search)

In [7]:
# Loop para encontrar o melhor modelo para cada 'Sub_Prefeitura' e 'Zona'
for sub_prefeitura in Sub_Prefeituras[0:1]:
    print("Ajustando modelos para Sub-Prefeitura de " + sub_prefeitura)
    best_models_subpref = grid_search_best_model_no_parallel(df_subpref, 'Sub_Prefeitura', sub_prefeitura)
    with open(f'best_models_subpref_{sub_prefeitura}.pkl', 'wb') as f:
        pickle.dump(best_models_subpref, f)

print("Modelos treinados e salvos com sucesso.")        

Ajustando modelos para Sub-Prefeitura de ARICANDUVA-FORMOSA-CARRAO
O intevalo corresponde a 123  datas
Séries geradas para o modelo de séries temporais
Modelos treinados e salvos com sucesso.


In [ ]:
for zona in Zonas[0:1]:
    print("Ajustando modelos para Zona " + zona)
    best_models_zona = grid_search_best_model(df_zonas, 'Zona', zona)
    with open(f'best_models_zona_{zona}.pkl', 'wb') as f:
        pickle.dump(best_models_zona, f)

print("Modelos treinados e salvos com sucesso.")

### DEBBUG

In [10]:
import pickle

# Carregar os modelos a partir do arquivo pickle
with open('best_models_subpref_ARICANDUVA-FORMOSA-CARRAO.pkl', 'rb') as f:
    loaded_models = pickle.load(f)

# Acessar o modelo SimpleExpSmoothing
simple_exp_smoothing_model = loaded_models['SimpleExpSmoothing']

In [11]:
print(simple_exp_smoothing_model.summary())

                               SimpleExpSmoothing Model Results                               
Dep. Variable:     Volume_Passageiros (BU + VT + DIN)   No. Observations:                  119
Model:                             SimpleExpSmoothing   SSE                    40462718989.207
Optimized:                                       True   AIC                           2341.698
Trend:                                           None   BIC                           2347.257
Seasonal:                                        None   AICC                          2342.049
Seasonal Periods:                                None   Date:                 Wed, 05 Mar 2025
Box-Cox:                                        False   Time:                         18:33:58
Box-Cox Coeff.:                                  None                                         
                       coeff                 code              optimized      
------------------------------------------------------------------

In [ ]:
simple_exp_smoothing_model.forecast(12)

In [ ]:


forecast = simple_exp_smoothing_model.forecast(steps=len(y_teste))
mape = mean_absolute_percentage_error(y_teste, forecast)
print(f"MAPE: {mape}")

In [ ]:
from joblib import Parallel, delayed
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

In [12]:
Serie_Historica, Serie_Treinamento, Serie_Teste = ajuste_datas_historica(df_subpref,
                                                                         coluna='Sub_Prefeitura',
                                                                         param='ARICANDUVA-FORMOSA-CARRAO')

  # Converte para arrays
y, y_treinamento, y_teste, y_treinamento_exp = conversao_array(Serie_Historica, Serie_Treinamento, Serie_Teste)

O intevalo corresponde a 123  datas
Séries geradas para o modelo de séries temporais


In [25]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import gc

In [33]:
# Função para verificar a estacionaridade
def check_stationarity(y):
    result = adfuller(y)
    return result[1] < 0.05

# Função para realizar o grid search e encontrar o melhor modelo
def grid_search_best_model(df, 
                           column, 
                           param):
    # Normaliza as datas e divide em conjuntos de treinamento e teste
    Serie_Historica, Serie_Treinamento, Serie_Teste = ajuste_datas_historica(df, coluna=column, param=param)
    
    # Converte para arrays
    y, y_treinamento, y_teste, y_treinamento_exp = conversao_array(Serie_Historica, Serie_Treinamento, Serie_Teste)
    
    # Verifica a estacionaridade e diferencia se necessário
    if not check_stationarity(y_treinamento):
        y_treinamento = y_treinamento.diff().dropna()
        y_teste = y_teste.diff().dropna()
    
    
    best_models = {}
    
    # Função auxiliar para grid search
    def evaluate_model(model, y_treinamento, y_teste):
        gc.collect() 
        forecast = model.forecast(len(y_teste))
        mape = mean_absolute_percentage_error(y_teste, forecast)
        return mape, model
    
    # Grid search para Simple Exponential Smoothing
    gc.collect()  # Coleta de lixo para liberar memória    
    results = []
    for alpha in np.arange(0.1, 2.1, 0.1):
        model = SimpleExpSmoothing(y_treinamento).fit(smoothing_level=alpha)
        mape, fitted_model = evaluate_model(model, y_treinamento, y_teste)
        results.append((mape, fitted_model))
        print(f"Alpha: {alpha}, MAPE: {mape}, Params: {fitted_model.params}")
    best_mape, best_model = min(results, key=lambda x: x[0])
    best_models['SimpleExpSmoothing'] = best_model
    gc.collect()  # Coleta de lixo para liberar memória
    
    # Grid search para Exponential Smoothing
    results = Parallel(n_jobs=-1)(delayed(evaluate_model)(ExponentialSmoothing(y_treinamento, trend='additive', seasonal='additive', seasonal_periods=12).fit(smoothing_level=alpha), y_treinamento, y_teste) for alpha in np.arange(0.1, 2.1, 0.1))
    best_mape, best_model = min(results, key=lambda x: x[0])
    best_models['ExponentialSmoothing'] = best_model
    
    # Grid search para Holt-Winters
    results = Parallel(n_jobs=-1)(delayed(evaluate_model)(Holt(y_treinamento).fit(smoothing_level=alpha), y_treinamento, y_teste) for alpha in np.arange(0.1, 2.1, 0.1))
    best_mape, best_model = min(results, key=lambda x: x[0])
    best_models['HoltWinters'] = best_model
    
    # Grid search para ARIMA e SARIMA
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]
    
    # Melhor ARMA
    results = Parallel(n_jobs=-1)(delayed(evaluate_model)(ARIMA(y_treinamento, order=(param[0], 0, param[2])).fit(), y_treinamento, y_teste) for param in pdq)
    best_mape, best_model = min(results, key=lambda x: x[0])
    best_models['ARMA'] = best_model    
    

    def evaluate_arima(param, y_treinamento, y_teste):
        try:
            model = ARIMA(y_treinamento, 
                          order=param,
                          enforce_stationarity=True,
                          enforce_invertibility=False).fit()
            
            forecast = model.forecast(len(y_teste))
            mape = mean_absolute_percentage_error(y_teste, forecast)
            return mape, model
        except (LinAlgError, ValueError):
            return float('inf'), None
    
    # Melhor ARIMA
    results = Parallel(n_jobs=-1)(delayed(evaluate_arima)(param,y_treinamento, y_teste) for param in pdq)
    best_mape, best_model = min(results, key=lambda x: x[0])
    best_models['ARIMA'] = best_model
    
    
    def evaluate_sarima(param, param_seasonal, y_treinamento, y_teste):
        try:
            model = sm.tsa.statespace.SARIMAX(y_treinamento, 
                                              order=param, 
                                              seasonal_order=param_seasonal, 
                                              enforce_stationarity=True, enforce_invertibility=False).fit(disp=0)
            
            forecast = model.forecast(len(y_teste))
            mape = mean_absolute_percentage_error(y_teste, forecast)
            return mape, model
        except (LinAlgError, ValueError):
            return float('inf'), None
        
        
    # Melhor SARIMA
    results = Parallel(n_jobs=-1)(delayed(evaluate_sarima)(param, param_seasonal, y_treinamento, y_teste) for param in pdq for param_seasonal in seasonal_pdq)
    best_mape, best_model = min(results, key=lambda x: x[0])
    best_models['SARIMA'] = best_model    
    
    
    return best_models


In [34]:
result = grid_search_best_model(df_subpref, 'Sub_Prefeitura', 'ARICANDUVA-FORMOSA-CARRAO')

O intevalo corresponde a 123  datas
Séries geradas para o modelo de séries temporais
Alpha: 0.1, MAPE: 0.9845237002142171, Params: {'smoothing_level': 0.1, 'smoothing_trend': nan, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': -584.5750741839752, 'initial_trend': nan, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}
Alpha: 0.2, MAPE: 1.8247255736375116, Params: {'smoothing_level': 0.2, 'smoothing_trend': nan, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': -1569.7514120040446, 'initial_trend': nan, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}
Alpha: 0.30000000000000004, MAPE: 4.521212102239824, Params: {'smoothing_level': 0.30000000000000004, 'smoothing_trend': nan, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': -3086.596404255469, 'initial_trend': nan, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'l

In [35]:
result['SimpleExpSmoothing'].summary()

Dep. Variable:,Volume_Passageiros (BU + VT + DIN),No. Observations:,119
Model:,SimpleExpSmoothing,SSE,40462718989.207
Optimized:,True,AIC,2341.698
Trend:,None,BIC,2347.257
Seasonal:,None,AICC,2342.049
Seasonal Periods:,None,Date:,"Wed, 05 Mar 2025"
Box-Cox:,False,Time:,18:39:32
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.1000000,alpha,False
initial_level,-584.57507,l.0,True


In [32]:
print(result['SimpleExpSmoothing'].params)

{'smoothing_level': 0.1, 'smoothing_trend': nan, 'smoothing_seasonal': nan, 'damping_trend': nan, 'initial_level': -584.5750741839752, 'initial_trend': nan, 'initial_seasons': array([], dtype=float64), 'use_boxcox': False, 'lamda': None, 'remove_bias': False}


In [ ]:
best_models = {}
results = Parallel(n_jobs=-1)(delayed(evaluate_model)(SimpleExpSmoothing(y_treinamento).fit(smoothing_level=alpha), y_treinamento, y_teste) for alpha in np.arange(0.1, 2.1, 0.1))
best_mape, best_model = min(results, key=lambda x: x[0])

best_models['SimpleExpSmoothing'] = best_model

In [ ]:
print(best_models['SimpleExpSmoothing'].summary())

In [ ]:
best_models['SimpleExpSmoothing'].forecast(12)

In [ ]:
print(result['SimpleExpSmoothing'].summary())

In [ ]:
result['SimpleExpSmoothing'].forecast(12)

In [ ]:
forecast = result['SimpleExpSmoothing'].forecast(len(y_teste))
mape = mean_absolute_percentage_error(y_teste, forecast)
mape

In [ ]:
result['SimpleExpSmoothing'].

In [ ]:
forecast = best_models['SimpleExpSmoothing'].forecast(len(y_teste))
mape = mean_absolute_percentage_error(y_teste, forecast)
mape

In [ ]:
# Grid search para Simple Exponential Smoothing
results = Parallel(n_jobs=-1)(delayed(evaluate_model)(SimpleExpSmoothing(y_treinamento).fit(smoothing_level=alpha), y_treinamento, y_teste) for alpha in np.arange(0.05, 2.1, 0.05))
best_mape, best_model = min(results, key=lambda x: x[0])
#best_models['SimpleExpSmoothing'] = best_model

In [ ]:
best_mape, best_model = min(results, key=lambda x: x[0])

In [ ]:
results

In [ ]:
best_mape

In [ ]:
print(best_model.summary())

In [ ]:
best_model.predict(12)

In [ ]:
fcst = best_model.forecast(12)

In [ ]:
np.pad(fcst, (max_len - len(fcst), 0), 'constant', constant_values=np.nan)

In [ ]:
def load_models(file_path):
    '''
    Função utilizada para carregar os pickles 
    contendo os modelos de forecast
    '''    
    with open(file_path, 'rb') as f:
        models = pickle.load(f)
    return models

In [ ]:
# Dicionário com os caminhos dos arquivos pickle dos modelos
model_file_subpref = 'best_models_subpref_{0}.pkl'
model_file_zona = 'best_models_zona_{0}.pkl'

In [ ]:
param='ARICANDUVA-FORMOSA-CARRAO'
# Carregar os modelos a partir do arquivo pickle
models = load_models(model_file_subpref.format(param))

In [ ]:
# Fazer o forecast de 12 meses para cada modelo
print(f"Gerando forecasts")
forecasts = {model_name: model.forecast(steps=12) for model_name, model in models.items()}

In [ ]:
with open('best_models_subpref_ARICANDUVA-FORMOSA-CARRAO.pkl', 'wb') as f:
    pickle.dump(best_models, f)

In [ ]:
f